In [40]:
from docx2pdf import convert
import os
import pandas as pd


In [41]:
# colocar aqui la ruta de los archivos
path = r'C:\Users\Usuario\Desktop\PLANOS'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1647 INFORMES
1647 INFORMES


In [42]:
len(rutas)

1647

In [43]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [44]:
contenido = os.listdir(path)
dic_files = {}
for file in contenido:
    split1 = file.split('.')    
    split2 = split1[0].split('_')
    dic_files[str(split2[1])] = file
print(len(dic_files), 'informes')

1647 informes


In [47]:
llaves = list(dic_files.keys())

In [49]:
print(llaves)

['1', '10', '100', '1000', '1001', '1002', '1005', '1006', '1009', '1011', '1013', '1014', '1015', '1018', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1028', '103', '1033', '1034', '1035', '1036', '1038', '1039', '104', '1040', '1041', '1042', '1043', '1044', '1047', '1049', '1050', '1051', '1052', '1053', '1054', '1057', '106', '1064', '1065', '1066', '1067', '1069', '1070', '1072', '1073', '1074', '1075', '1076', '1077', '108', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1090', '1091', '1092', '1093', '1094', '1095', '1096', '1097', '1099', '11', '110', '1100', '1104', '1106', '1107', '1108', '1109', '111', '1111', '1113', '1114', '1115', '1116', '1117', '1118', '1119', '112', '1120', '1121', '1122', '1123', '1124', '1125', '1126', '1128', '113', '1131', '1136', '1137', '1139', '114', '1140', '1141', '1142', '1144', '1146', '1147', '1148', '1149', '1150', '1151', '1153', '1154', '1156', '1157', '1158', '1159', '116', '1160', '1161', '1163', '

In [ ]:
dic_urls = {}

In [77]:
llaves = ['589', '590', '591', '592', '593', '594', '597', '598', '6', '600', '601', '602', '603', '604', '605', '606', '61', '611', '613', '614', '615', '616', '617', '618', '619', '62', '620', '621', '622', '623', '625', '627', '628', '629', '63', '630', '631', '632', '633', '634', '636', '637', '638', '639', '640', '641', '642', '643', '645', '646', '647', '648', '649', '650', '651', '652', '653', '654', '655', '656', '657', '658', '659', '66', '660', '661', '664', '665', '666', '667', '668', '669', '67', '670', '671', '672', '673', '674', '675', '676', '677', '678', '679', '68', '681', '682', '683', '691', '693', '694', '695', '697', '699', '700', '702', '703', '704', '705', '706', '708', '709', '71', '710', '711', '714', '715', '716', '717', '718', '719', '72', '720', '721', '722', '723', '724', '725', '726', '727', '728', '729', '730', '732', '733', '734', '737', '739', '74', '741', '742', '743', '744', '745', '746', '747', '749', '750', '752', '753', '755', '756', '757', '758', '759', '76', '761', '762', '763', '764', '765', '766', '767', '768', '769', '770', '771', '772', '773', '774', '775', '776', '777', '778', '779', '78', '780', '781', '782', '783', '784', '785', '787', '788', '789', '79', '790', '791', '793', '794', '795', '796', '797', '798', '799', '8', '80', '800', '801', '802', '803', '804', '805', '806', '807', '808', '809', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '823', '824', '825', '827', '828', '829', '83', '830', '831', '832', '833', '834', '835', '84', '840', '841', '842', '843', '844', '845', '846', '848', '849', '85', '851', '852', '853', '854', '855', '856', '858', '859', '860', '861', '862', '863', '864', '865', '866', '867', '868', '869', '870', '871', '872', '873', '874', '875', '877', '878', '879', '880', '881', '882', '883', '886', '887', '888', '889', '89', '890', '891', '893', '894', '895', '896', '897', '898', '899', '9', '900', '901', '902', '903', '904', '905', '906', '907', '909', '910', '911', '912', '913', '915', '916', '918', '919', '92', '920', '921', '922', '923', '924', '925', '926', '927', '928', '929', '93', '930', '931', '932', '934', '935', '936', '937', '938', '939', '94', '940', '941', '942', '943', '944', '945', '947', '948', '949', '950', '951', '952', '953', '955', '956', '957', '958', '96', '961', '962', '963', '964', '966', '967', '968', '969', '970', '971', '972', '973', '974', '975', '976', '978', '979', '98', '980', '981', '982', '984', '988', '990', '991', '992', '993', '994', '995', '996', '999']

In [80]:
# CARGAR LOS PDFS A GOOGLE DRIVE
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "PLANOS ANOMALIAS"):
      fileID = file['id']

for file in llaves:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(path + r'/' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file PROPIEDAD_589.pdf with mimeType application/pdf
Created file PROPIEDAD_590.pdf with mimeType application/pdf
Created file PROPIEDAD_591.pdf with mimeType application/pdf
Created file PROPIEDAD_592.pdf with mimeType application/pdf
Created file PROPIEDAD_593.pdf with mimeType application/pdf
Created file PROPIEDAD_594.pdf with mimeType application/pdf
Created file PROPIEDAD_597.pdf with mimeType application/pdf
Created file PROPIEDAD_598.pdf with mimeType application/pdf
Created file PROPIEDAD_6.pdf with mimeType application/pdf
Created file PROPIEDAD_600.pdf with mimeType application/pdf
Created file PROPIEDAD_601.pdf with mimeType application/pdf
Created file PROPIEDAD_602.pdf with mimeType application/pdf
Created file PROPIEDAD_603.pdf with mimeType application/pdf
Created file PROPIEDAD_604.pdf with mimeType application/pdf
Created file PROPIEDAD_605.pdf with mimeType application/pdf
Created file PROPIEDAD_606.pdf with mimeType application/pdf
Created file PROPIEDAD_61.

In [81]:
file

'999'

In [1]:
len(dic_urls)

NameError: name 'dic_urls' is not defined

In [83]:
df = pd.DataFrame(list(dic_urls.items()), columns=['ID', 'URL'])

# Exportamos el DataFrame a un archivo Excel
df.to_excel('ruta_planos_5.xlsx', index=False)